In [1]:
import pyspark
from pyspark.sql import SparkSession

TypeError: code() argument 13 must be str, not int

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

23/03/15 18:02:13 WARN Utils: Your hostname, centaur.local resolves to a loopback address: 127.0.0.1; using 172.20.10.8 instead (on interface en0)
23/03/15 18:02:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/15 18:02:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/15 18:02:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [8]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2020-05.parquet

--2023-03-15 17:51:22--  https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2020-05.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.161.108.231, 18.161.108.184, 18.161.108.77, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.161.108.231|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 160544713 (153M) [application/x-www-form-urlencoded]
Saving to: ‘fhvhv_tripdata_2020-05.parquet’

fhvhv_tripdata_2020 100%[===================>] 153.11M  1.09MB/s    in 2m 19s  

2023-03-15 17:53:43 (1.10 MB/s) - ‘fhvhv_tripdata_2020-05.parquet’ saved [160544713/160544713]



In [3]:
!wc -l fhvhv_tripdata_2020-05.parquet

  516719 fhvhv_tripdata_2020-05.parquet


In [5]:
!pip install pandas
!pip install pyarrow
!pip install fastparquet

In [6]:
import pandas as pd
df = pd.read_parquet('fhvhv_tripdata_2020-05.parquet')
df.to_csv('fhvhv_tripdata_2020-05.csv')

In [7]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2020-05.csv')

In [8]:
df.schema

StructType([StructField('_c0', StringType(), True), StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('originating_base_num', StringType(), True), StructField('request_datetime', StringType(), True), StructField('on_scene_datetime', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('trip_miles', StringType(), True), StructField('trip_time', StringType(), True), StructField('base_passenger_fare', StringType(), True), StructField('tolls', StringType(), True), StructField('bcf', StringType(), True), StructField('sales_tax', StringType(), True), StructField('congestion_surcharge', StringType(), True), StructField('airport_fee', StringType(), True), StructField('tips', StringType(), True), StructField('driver_pay', StringType(), True), 

In [9]:
!head -n 1001 fhvhv_tripdata_2020-05.csv > head.csv

In [10]:
df_pandas = pd.read_csv('head.csv')

In [11]:
df_pandas.dtypes

Unnamed: 0                int64
hvfhs_license_num        object
dispatching_base_num     object
originating_base_num     object
request_datetime         object
on_scene_datetime        object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
trip_miles              float64
trip_time                 int64
base_passenger_fare     float64
tolls                   float64
bcf                     float64
sales_tax               float64
congestion_surcharge    float64
airport_fee             float64
tips                    float64
driver_pay              float64
shared_request_flag      object
shared_match_flag        object
access_a_ride_flag       object
wav_request_flag         object
wav_match_flag           object
dtype: object

In [14]:
df_pandas.isnull().sum()

Unnamed: 0                 0
hvfhs_license_num          0
dispatching_base_num       0
originating_base_num     247
request_datetime           0
on_scene_datetime        247
pickup_datetime            0
dropoff_datetime           0
PULocationID               0
DOLocationID               0
trip_miles                 0
trip_time                  0
base_passenger_fare        0
tolls                      0
bcf                        0
sales_tax                  0
congestion_surcharge       0
airport_fee             1000
tips                       0
driver_pay                 0
shared_request_flag        0
shared_match_flag          0
access_a_ride_flag         0
wav_request_flag           0
wav_match_flag             0
dtype: int64

In [19]:
df_pandas["originating_base_num"] = df_pandas["originating_base_num"].fillna("Nan values")
df_pandas["on_scene_datetime"] = df_pandas["on_scene_datetime"].fillna("Nan values")
df_pandas["airport_fee"] = df_pandas["airport_fee"].fillna("Nan values")
df_pandas.isnull().sum()

Unnamed: 0              0
hvfhs_license_num       0
dispatching_base_num    0
originating_base_num    0
request_datetime        0
on_scene_datetime       0
pickup_datetime         0
dropoff_datetime        0
PULocationID            0
DOLocationID            0
trip_miles              0
trip_time               0
base_passenger_fare     0
tolls                   0
bcf                     0
sales_tax               0
congestion_surcharge    0
airport_fee             0
tips                    0
driver_pay              0
shared_request_flag     0
shared_match_flag       0
access_a_ride_flag      0
wav_request_flag        0
wav_match_flag          0
dtype: int64

In [35]:
spark.createDataFrame(df_pandas).schema

StructType([StructField('Unnamed: 0', LongType(), True), StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('originating_base_num', StringType(), True), StructField('request_datetime', StringType(), True), StructField('on_scene_datetime', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('trip_miles', DoubleType(), True), StructField('trip_time', LongType(), True), StructField('base_passenger_fare', DoubleType(), True), StructField('tolls', DoubleType(), True), StructField('bcf', DoubleType(), True), StructField('sales_tax', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', StringType(), True), StructField('tips', DoubleType(), True), StructField('driver_pay', DoubleType(), True), S

Integer - 4 bytes
Long - 8 bytes

In [29]:
from pyspark.sql import types

In [45]:
schema = types.StructType([
    types.StructField('', types.IntegerType(), True),
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('originating_base_num', types.StringType(), True),
    types.StructField('request_datetime', types.StringType(), True),
    types.StructField('on_scene_datetime', types.StringType(), True),
    types.StructField('pickup_datetime', types.StringType(), True),
    types.StructField('dropoff_datetime', types.StringType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('trip_miles', types.FloatType(), True),
    types.StructField('trip_time', types.IntegerType(), True),
    types.StructField('base_passenger_fare', types.FloatType(), True),
    types.StructField('tolls', types.FloatType(), True),
    types.StructField('bcf', types.FloatType(), True),
    types.StructField('sales_tax', types.FloatType(), True),
    types.StructField('congestion_surcharge', types.FloatType(), True),
    types.StructField('airport_fee', types.FloatType(), True),
    types.StructField('tips', types.FloatType(), True),
    types.StructField('driver_pay', types.FloatType(), True),
    types.StructField('shared_request_flag', types.StringType(), True),
    types.StructField('shared_match_flag', types.StringType(), True),
    types.StructField('access_a_ride_flag', types.StringType(), True),
    types.StructField('wav_request_flag', types.StringType(), True),
    types.StructField('wav_match_flag', types.StringType(), True)
])

In [46]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2020-05.csv')

In [47]:
df = df.repartition(24)

In [48]:
df.write.parquet('fhvhv/2020/05/')

23/03/15 18:26:18 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/03/15 18:26:18 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers


23/03/15 18:26:20 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers


23/03/15 18:26:20 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/03/15 18:26:20 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/03/15 18:26:21 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers


23/03/15 18:26:21 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/03/15 18:26:21 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers


23/03/15 18:26:22 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers


In [49]:
df = spark.read.parquet('fhvhv/2020/05/')

In [50]:
df.printSchema()

root
 |-- : integer (nullable = true)
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: string (nullable = true)
 |-- on_scene_datetime: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- trip_miles: float (nullable = true)
 |-- trip_time: integer (nullable = true)
 |-- base_passenger_fare: float (nullable = true)
 |-- tolls: float (nullable = true)
 |-- bcf: float (nullable = true)
 |-- sales_tax: float (nullable = true)
 |-- congestion_surcharge: float (nullable = true)
 |-- airport_fee: float (nullable = true)
 |-- tips: float (nullable = true)
 |-- driver_pay: float (nullable = true)
 |-- shared_request_flag: string (nullable = true)
 |-- shared_match_flag: string (nullable = true)
 |-- access_a_r

SELECT * FROM df WHERE hvfhs_license_num =  HV0003

In [51]:
from pyspark.sql import functions as F

In [52]:
df.show()

+------+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|      |hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+------+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------

In [53]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [54]:
crazy_stuff('B02884')

's/b44'

In [55]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [56]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  e/b38| 2020-05-03|  2020-05-03|          75|          74|
|  e/b47| 2020-05-02|  2020-05-02|         136|         208|
|  s/b36| 2020-05-01|  2020-05-01|         177|          37|
|  e/b3f| 2020-05-01|  2020-05-01|         142|          18|
|  e/b30| 2020-05-05|  2020-05-05|          77|          65|
|  e/a39| 2020-05-05|  2020-05-05|          36|          37|
|  e/9ce| 2020-05-01|  2020-05-01|          89|          65|
|  e/b38| 2020-05-02|  2020-05-02|         225|         177|
|  s/b13| 2020-05-05|  2020-05-05|          61|         181|
|  e/b33| 2020-05-02|  2020-05-02|         203|         265|
|  e/9ce| 2020-05-02|  2020-05-02|          51|         254|
|  e/acc| 2020-05-03|  2020-05-03|          82|         226|
|  e/b32| 2020-05-01|  2020-05-01|         169|         247|
|  a/b37| 2020-05-03|  2

In [57]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
  .filter(df.hvfhs_license_num == 'HV0003')


DataFrame[pickup_datetime: string, dropoff_datetime: string, PULocationID: int, DOLocationID: int]

In [58]:
!head -n 10 head.csv

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,HV0003,B02764,B02764,2020-05-01 00:34:06,2020-05-01 00:40:12,2020-05-01 00:41:02,2020-05-01 01:08:04,142,69,8.85,1622,31.11,0.0,0.79,2.8,2.75,,0.0,23.34,N,N, ,N,N
1,HV0003,B02764,B02764,2020-05-01 00:06:47,2020-05-01 00:10:34,2020-05-01 00:13:27,2020-05-01 00:27:20,126,254,6.35,833,19.05,0.0,0.48,1.69,0.0,,0.0,13.97,N,N, ,N,N
2,HV0003,B02764,B02764,2020-05-01 00:27:19,2020-05-01 00:27:32,2020-05-01 00:29:50,2020-05-01 01:01:52,254,265,18.85,1922,55.1,6.12,1.53,5.43,0.0,,0.0,40.63,N,N, ,N,N
3,HV0003,B02875,B02875,2020-05-01 00:20:01,2020-05-01 00:24:15,2020-05-01 00:24:47,2020-05-01 00:56:04,170,265,19.35,1877,51.85,6.12,1.62,5.16,2.75,,